In [ ]:
#!pip install -q pyomo
#!pip install gurobipy

import pyomo.environ as pyo
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 27.8 MB/s eta 0:00:00


In [ ]:
# sets
SCHOOLS = ['Architecture',
           'Arts and Media',
           'Business',
           'Engineering',
           'Public Affairs',
           'Liberal Arts and Sciences',
           'Education and Human Development']

# parameters

s = {'Architecture' : 450,
    'Arts and Media' : 1393,
    'Business' : 2408,
    'Engineering' : 1277,
    'Public Affairs' : 493,
    'Liberal Arts and Sciences' : 8421,
    'Education and Human Development' : 247}

t = {'Architecture' : 25,
    'Arts and Media' : 77,
    'Business' : 134,
    'Engineering' : 71,
    'Public Affairs' : 27,
    'Liberal Arts and Sciences' : 458,
    'Education and Human Development' : 14}

p = {'Architecture' : 95000,
    'Arts and Media' : 70000,
    'Business' : 101000,
    'Engineering' : 109000,
    'Public Affairs' : 84000,
    'Liberal Arts and Sciences' : 92000,
    'Education and Human Development' : 73000}

c = {'Architecture' : 7,
    'Arts and Media' : 7,
    'Business' : 10,
    'Engineering' : 9,
    'Public Affairs' : 7,
    'Liberal Arts and Sciences' : 8,
    'Education and Human Development' : 9}

maxRatio = 18
maxBudget = 180000000  # budget from https://www.cu.edu/doc/2022-02-11-fy2022-23-budget-and-fee-proposals-fullpdf p.72
min_credits = 6672

In [ ]:
model = pyo.ConcreteModel()

# Variables
model.x = pyo.Var(SCHOOLS, domain=pyo.NonNegativeIntegers)

# objective function
def obj_rule(model):
    return sum((model.x[i] + t[i]) for i in SCHOOLS)
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# constraints
def ratio_rule(model, i):
    return (maxRatio * (model.x[i] + t[i])) >= s[i]
model.ratio_const = pyo.Constraint(s.keys(), rule=ratio_rule)

def max_budget_rule(model):
    return sum(p[i]*model.x[i] for i in SCHOOLS) <= maxBudget
model.max_budget_constraint = pyo.Constraint(rule=max_budget_rule)

def min_credit_rule(model):
    return sum(c[i]*(model.x[i] + t[i]) for i in SCHOOLS) >= min_credits
model.min_class_constraint = pyo.Constraint(rule=min_credit_rule)

In [ ]:
# solve the model verbosely
# result = pyo.SolverFactory('gurobi').solve(model, tee=True)

# solve the model quietly
result = pyo.SolverFactory('gurobi').solve(model)

# print results
# check if model solved to optimality then print results
if (result.solver.termination_condition == pyo.TerminationCondition.optimal):
    print(f'Total Teachers = {pyo.value(model.obj)}')
    for i in SCHOOLS:
        print(f'Optimal number of {i} teachers to hire is {pyo.value(model.x[i])}')
else:
    print(f'Solver termination condition: {result.solver.termination_condition}')

Total Teachers = 818.0
Optimal number of Architecture teachers to hire is -0.0
Optimal number of Arts and Media teachers to hire is 1.0
Optimal number of Business teachers to hire is -0.0
Optimal number of Engineering teachers to hire is -0.0
Optimal number of Public Affairs teachers to hire is 1.0
Optimal number of Liberal Arts and Sciences teachers to hire is 10.0
Optimal number of Education and Human Development teachers to hire is -0.0
